In [ ]:
!pip uninstall -y langchain langchain-openai
!pip install -q langchain langchain-openai

Found existing installation: langchain 1.2.10
Uninstalling langchain-1.2.10:
  Successfully uninstalled langchain-1.2.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

In [ ]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluates a mathematical expression.
    Example: "2 + 2 * 10"
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception:
        return "Invalid mathematical expression."

In [ ]:
@tool
def get_order_status(order_id: str) -> str:
    """
    Returns order status for a given order ID.
    """
    fake_database = {
        "ORD123": "Shipped",
        "ORD456": "Processing",
        "ORD789": "Delivered"
    }
    return fake_database.get(order_id, "Order not found.")

In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Give me a short definition of DNN."}
    ]
)

print(response.choices[0].message.content)

A Deep Neural Network (DNN) is a type of artificial neural network with multiple layers of nodes (neurons) between the input and output layers, allowing it to learn complex patterns and representations from large datasets. DNNs are commonly used in various applications such as image and speech recognition, natural language processing, and more.


In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

tools = [calculator, get_order_status]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
query = "What is 45 multiplied by 12?"

In [ ]:
response = llm_with_tools.invoke(
    [HumanMessage(content=query)]
)

response

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 87, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DDhT3raan1OJl0mC1VKecCAkbzrlX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c9cd4-e896-7870-bd82-14c835556026-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '45 * 12'}, 'id': 'call_dVK54lKlc9lkCCvAKWmkZZQl', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 87, 'output_tokens': 17, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
def run_with_tools(user_query):
    messages = [HumanMessage(content=user_query)]

    while True:
        response = llm_with_tools.invoke(messages)

        # If no tool call → return final answer
        if not response.tool_calls:
            return response.content

        messages.append(response)

        # Execute each tool call
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            if tool_name == "get_order_status":
                tool_result = get_order_status.invoke(tool_args)

            if tool_name == "calculator":
                tool_result = calculator.invoke(tool_args)

                messages.append(
                    ToolMessage(
                        content=tool_result,
                        tool_call_id=tool_call["id"]
                    )
                )

In [ ]:
answer = run_with_tools("What is 45 multiplied by 12?")
print(answer)

45 multiplied by 12 is 540.


### **Auto Tool Selection with LangChain**

In [ ]:
!pip uninstall -y langchain langchain-openai requests
!pip install -q langchain langchain-openai requests

Found existing installation: langchain 1.2.10
Uninstalling langchain-1.2.10:
  Successfully uninstalled langchain-1.2.10
Found existing installation: langchain-openai 1.1.10
Uninstalling langchain-openai-1.1.10:
  Successfully uninstalled langchain-openai-1.1.10
Found existing installation: requests 2.32.4
Uninstalling requests-2.32.4:
  Successfully uninstalled requests-2.32.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import requests
from datetime import datetime

In [ ]:
OPENWEATHER_API_KEY = "209688131a0268b51c101f612dca54ca"

In [ ]:
@tool
def get_weather(city: str) -> str:
    """Get current weather for a given city."""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url).json()

    if response.get("cod") != 200:
        return "City not found."

    temp = response["main"]["temp"]
    description = response["weather"][0]["description"]
    return f"The current temperature in {city} is {temp}°C with {description}."

In [ ]:
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        return str(eval(expression))
    except:
        return "Invalid expression."

In [ ]:
@tool
def calculate_interest(principal: float, rate: float, years: float) -> str:
    """Calculate simple interest given principal, annual rate (%), and years."""
    interest = (principal * rate * years) / 100
    total = principal + interest
    return f"Interest: {interest}, Total amount: {total}"

In [ ]:
@tool
def get_current_date() -> str:
    """Returns today's date."""
    return datetime.now().strftime("%Y-%m-%d")

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

tools = [
    get_weather,
    calculator,
    calculate_interest,
    get_current_date
]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
def run_agent(user_query):
    messages = [HumanMessage(content=user_query)]

    while True:
        response = llm_with_tools.invoke(messages)

        # If no tool call, return answer
        if not response.tool_calls:
            return response.content

        messages.append(response)

        # Execute tool calls
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            tool_map = {
                "get_weather": get_weather,
                "calculator": calculator,
                "calculate_interest": calculate_interest,
                "get_current_date": get_current_date
            }

            tool_result = tool_map[tool_name].invoke(tool_args)

            messages.append(
                ToolMessage(
                    content=tool_result,
                    tool_call_id=tool_call["id"]
                )
            )

In [ ]:
run_agent("What is the weather in Mumbai?")

'The current temperature in Mumbai is 23.99°C with haze.'

In [ ]:
run_agent("What is 245 multiplied by 18?")

'245 multiplied by 18 is 4410.'

In [ ]:
run_agent("Calculate interest for 100000 at 7% for 3 years")

'The interest for a principal of $100,000 at an annual rate of 7% for 3 years is $21,000. The total amount after 3 years will be $121,000.'

In [ ]:
run_agent("What is today's date?")

"Today's date is February 27, 2026."

In [ ]:
run_agent("What is Machine Learning")

"Machine Learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. Instead of being programmed to perform a task, machine learning systems learn from data, identify patterns, and make decisions based on that data.\n\nKey concepts in machine learning include:\n\n1. **Data**: The foundation of machine learning. Models are trained on large datasets to learn patterns and make predictions.\n\n2. **Algorithms**: These are the mathematical procedures that process the data. Common algorithms include decision trees, neural networks, support vector machines, and clustering algorithms.\n\n3. **Training**: The process of feeding data into a machine learning model to help it learn. This involves adjusting the model's parameters to minimize errors in its predictions.\n\n4. **Testing and Validation**: After training, the model is tested on unseen data to 

In [ ]:
SERPER_API_KEY = "c1da5517f1e790b5ae275a6fc44e2a47bf6fa618"

In [ ]:
@tool
def web_search(query: str) -> str:
    """Search the web for up-to-date information using Google search."""

    url = "https://google.serper.dev/search"

    headers = {
        "X-API-KEY": SERPER_API_KEY,
        "Content-Type": "application/json"
    }

    payload = {
        "q": query,
        "num": 5
    }

    response = requests.post(url, headers=headers, json=payload).json()

    if "organic" not in response:
        return "No results found."

    results = []
    for item in response["organic"][:3]:
        title = item.get("title", "")
        snippet = item.get("snippet", "")
        link = item.get("link", "")
        results.append(f"{title}\n{snippet}\n{link}\n")

    return "\n".join(results)

In [ ]:
def run_agent(user_query):
    messages = [HumanMessage(content=user_query)]

    while True:
        response = llm_with_tools.invoke(messages)

        # If no tool call → return final answer
        if not response.tool_calls:
            return response.content

        messages.append(response)

        # Execute tool calls
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Updated tool map (added web_search)
            tool_map = {
                "get_weather": get_weather,
                "calculator": calculator,
                "calculate_interest": calculate_interest,
                "get_current_date": get_current_date,
                "web_search": web_search   # <-- Added here
            }

            # Safety check (recommended)
            if tool_name not in tool_map:
                raise ValueError(f"Unknown tool requested: {tool_name}")

            tool_result = tool_map[tool_name].invoke(tool_args)

            messages.append(
                ToolMessage(
                    content=tool_result,
                    tool_call_id=tool_call["id"]
                )
            )

In [ ]:
run_agent("Who won the most recent ICC Cricket World Cup?")

'Australia won the most recent ICC Cricket World Cup, which took place on November 19, 2023. They defeated India by 6 wickets in the final match held at the Narendra Modi Stadium in Ahmedabad.'

In [ ]:
def run_agent(user_query):
    messages = [HumanMessage(content=user_query)]

    print(f"\n🧑 User: {user_query}")

    while True:
        response = llm_with_tools.invoke(messages)

        # ✅ If no tool call → pure LLM response
        if not response.tool_calls:
            print("\n🤖 Model Response (No tool used)")
            print(response.content)
            return response.content

        messages.append(response)

        # Execute tool calls
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            print(f"\n🔧 Tool Selected: {tool_name}")
            print(f"📦 Tool Arguments: {tool_args}")

            tool_map = {
                "get_weather": get_weather,
                "calculator": calculator,
                "calculate_interest": calculate_interest,
                "get_current_date": get_current_date,
                "web_search": web_search
            }

            if tool_name not in tool_map:
                raise ValueError(f"Unknown tool requested: {tool_name}")

            tool_result = tool_map[tool_name].invoke(tool_args)

            print(f"📤 Tool Result:\n{tool_result}")

            messages.append(
                ToolMessage(
                    content=tool_result,
                    tool_call_id=tool_call["id"]
                )
            )

In [ ]:
run_agent("What is the weather in Mumbai?")


🧑 User: What is the weather in Mumbai?

🔧 Tool Selected: get_weather
📦 Tool Arguments: {'city': 'Mumbai'}
📤 Tool Result:
The current temperature in Mumbai is 23.99°C with haze.

🤖 Model Response (No tool used)
The current weather in Mumbai is 23.99°C with haze.


'The current weather in Mumbai is 23.99°C with haze.'

In [ ]:
run_agent("Explain what machine learning is.")


🧑 User: Explain what machine learning is.

🤖 Model Response (No tool used)
Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. Instead of being programmed to perform a task, machine learning systems learn from data, identifying patterns and making decisions based on that data.

Here are some key concepts related to machine learning:

1. **Data**: Machine learning relies on large amounts of data to train models. This data can be structured (like databases) or unstructured (like images or text).

2. **Algorithms**: These are the mathematical models that process the data. Common algorithms include decision trees, neural networks, support vector machines, and clustering algorithms.

3. **Training**: The process of feeding data into a machine learning model so that it can learn from it. During training, the model adjusts its parameters

'Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. Instead of being programmed to perform a task, machine learning systems learn from data, identifying patterns and making decisions based on that data.\n\nHere are some key concepts related to machine learning:\n\n1. **Data**: Machine learning relies on large amounts of data to train models. This data can be structured (like databases) or unstructured (like images or text).\n\n2. **Algorithms**: These are the mathematical models that process the data. Common algorithms include decision trees, neural networks, support vector machines, and clustering algorithms.\n\n3. **Training**: The process of feeding data into a machine learning model so that it can learn from it. During training, the model adjusts its parameters to minimize errors in its predictions.\n\n4. **Testing and Validat